## Monte Carlo simulations of Lennard-Jones fluids

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import ipywidgets as widgets

# Load basic science tools
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
from numpy.random import rand, randint, uniform, choice, normal 

In [3]:
#! pip install plotly
#! pip install numba

In [6]:
# Install Plotly and numba for aswesome speed and visualizations
import plotly.express as px
from numba import jit, njit

import warnings
warnings.filterwarnings('ignore')

### Check out LJ fluid properties tabulated by NIST for your reference

https://www.nist.gov/mml/csd/chemical-informatics-research-group/lennard-jones-fluid-properties

In [15]:
#-------LJ potential parameters ----

sigma   = 1
epsilon = 1
trunc   = 3*sigma


#-------Simulation parameters ----

N        = 100      # Try different values
T        = 2.0      # Try different values
rho      = 0.85     # We set density and

L  =  (N/rho)**(1/3) 

In [53]:
def initialize(N, rho):
    
    '''Initialize the simulation box 
    given N  particles with density
    returns (N,3) array positions
    '''
    
    L  =  (N/rho)**(1/3) 

    return uniform(0, L, size=(N,3))

In [52]:
def initialize2(N, rho):
    
    """put N particles in a box, using dense packing unit lattice size n3"""
    
    L  =  (N/rho)**(1/3) 
    n3 = int(N **(1/3)) + 1
    iix = iiy = iiz = 0
    
    pos = np.zeros((N,3))
    for i in range(N):
        
        pos[i][0] = (iix + 0.5) * L / n3
        pos[i][1] = (iiy + 0.5) * L / n3
        pos[i][2] = (iiz + 0.5) * L / n3
        
        iix += 1
        
        if iix == n3:
            
            iix  = 0
            iiy += 1
            
            if iiy == n3:
                iiy = 0
                iiz += 1 
    return pos

In [57]:
pp = initialize2(N, rho)
#px.scatter_3d(x=pp[:,0], y=pp[:,1], z=pp[:,2], opacity=0.5)

In [71]:
@njit
def get_r2(p1, p2, L):
    
    ''' Compute squared distance between two particles p1 and p2 with PBC.'''
    dr = p2 - p1  
    
    for i in range(3):
        
        if dr[i] >  L/2:
            
            dr[i] = dr[i] - L
            
        if dr[i] < -L/2:
            
            dr[i] = dr[i] + L
        
    return np.sum(dr**2)

@njit
def getEij(pos_i, pos_j, trunc):
    
    pair_e  = 0
    
    dist_sq = get_r2(pos_i, pos_j, L)
                
    if dist_sq <= trunc**2:
                
        pair_e += (1/dist_sq**6)-(1/dist_sq**3)
        
    return 4*pair_e

@jit
def getE(particles, L, trunc, mode = 'all'):
    
    N = len(particles)
    energy = 0
    
    #Compute the energy of the system given particle positions
    if mode  == 'all':
    
        for i in range(0, N-1):
            
            for j in range(i+1, N):
            
                    energy += getEij(particles[i], particles[j], L)
                    
        return energy
    
    
    #Compute the energy of a single particle j
    else:
        
        j, pos_j = mode 
        all_j    = np.arange(N)[np.arange(N)!=j]  #Excluding j from particle indeces 
        
        for i in all_j:
                
                energy += getEij(particles[i], pos_j, L)
    
        return energy

### MC engine for LJ fluid in 3D (NVT ensemble)
Now that the main helper functions are set up we can put together a main Monte Carlo engine that loops through randomly selected particles and attemps their displacement via Metropolis Criterion. 

In [72]:
@jit
def doMC_LJ(particles, L, T, steps=10000, freq=1000):

    N      = len(particles)
    E_tot  = getE(particles, L, trunc, mode='all') 
    
    confs, es   = [particles.copy()], [E_tot]
    
    # Loop through MC steps 
    for step in range(0, steps):
        
        # Randomly choose some particle i in  
        i = randint(N)
    
        #Position and Energy of particle i
        pos_i      = particles[i]
        
        eng_i      = getE(particles, L, trunc, mode=(i, pos_i))
    
        #Give a particle i a random push:
        pos_i_new  = particles[i] + (rand(3)-0.5)
        
        eng_i_new  = getE(particles, L, trunc, mode=(i, pos_i_new))
        
        # Evaluate change in energy
        dE = eng_i_new - eng_i

        # Metropolis acceptance/rejection conditions
        
        if  np.exp(-dE/T) > rand():
            
            particles[i] = pos_i_new
            E_tot       += dE
            
        if step % freq ==0:
            
            confs.append(particles.copy())
            es.append(E_tot)
                    
    return confs, es 

In [73]:
particles = initialize2(N, rho)

%time confs, es = doMC_LJ(particles, L, T, steps=200000)

CPU times: user 24.9 s, sys: 238 ms, total: 25.1 s
Wall time: 26.4 s


In [66]:
#plt.plot(es)

In [67]:
i = 80
#px.scatter_3d(x=confs[i][:,0], y=confs[i][:,1], z=confs[i][:,2], opacity=0.5)

### Computing radial distribution function $g(r)$

In [260]:
@jit
def comp_g_r(particles, L):
    '''Computing radial distirbution function for a single configuration'''
    
    # Number of particles and density
    
    N   = len(particles)
    rho = N/(L**3)
    r2 = [] 
    
    for i in range(0,N-1):
        
        for j in range(i+1,N):

            dist_ij  = get_r2(particles[i],particles[j], L) 
            
            r2.append(dist_ij)

    r_dists = np.sqrt(r2)
    
    #Histogram distances 
    histRadii, edges = np.histogram(r_dists, bins=30, range=(0,L/2))
    
    radii = 0.5 *(edges[1:] + edges[:-1])  # radii of histograms R1,R2,..Lmax 
    
    # often folks loop over N*N pairs then divide by N
    # we have N*N-1/2 pairs e.g must divide by (N-1)/2 to have hist for N particles
    histRadii = histRadii/(0.5*(N-1))    
    
    #Compute histogram that would be displayed by an ideal gas of N particles 
    vol = (4.0/3.0) * np.pi * (edges[1:]**3 - edges[:-1]**3)
    
    histIdeal = rho*vol 
    
    gr = histRadii/histIdeal 
    
    return radii, gr

In [ ]:
# Simulation for computing g_r
N     = 100
rho   = 0.85
T     = 2 
steps = 100000
freq  = 200

######
particles       = initialize(N, rho)
%time confs, es = doMC_LJ(particles, L, T, steps, freq)
######

In [63]:
#n_pt    = int(steps/freq)
#i = n_pt -1
#px.scatter_3d(x=confs[i][:,0], y=confs[i][:,1], z=confs[i][:,2], opacity=0.5)

#### Calculate $g(r)$ over trajectory

In [463]:
gr_t = 0


av_wind = 1000

for i in range(n_pt-av_wind, n_pt):
    
    radii, gr = comp_g_r(confs[i],L)
    
    gr_t += gr/av_wind

In [62]:
#plt.plot(radii, gr_t)